In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from os import path
import time 
from datetime import datetime 
import math

import sys
import os
import subprocess
import re
import cv2
import mimetypes
import requests
import re

In [60]:
data_dir = path.join('..', 'data')
img_dir = path.join(data_dir, 'img')
temp_dir = path.join(data_dir, 'temp')

dataset_file = 'reddit_wsb_art.csv'

data = pd.read_csv(path.join(data_dir, dataset_file))
data = data.sort_values(by=['timestamp'])

In [61]:
RE_HTTP = re.compile("http(s)?://[/\.A-z0-9]+")

def detect_urls(text):
    text = str(text)

    return [str(x[1].group(0)) for x in enumerate(re.finditer(RE_HTTP, text))]

data['body_url'] = data.apply(lambda x: detect_urls(x['body']), axis=1) 
data['body_urls_count'] = data['body_url'].apply(len)

data[['body', 'body_url', 'body_urls_count']].loc[data['body_urls_count'] > 0]

,body,body_url,body_urls_count
11340,"When I saw the video I was in shock, I could n...","[https://www.marketwatch.com/story/gamestop, h...",7
21116,Intel posted earnings today of $1.52 EPS beati...,"[https://preview.redd.it/2lqwp6j4urc61.png, ht...",3
10187,I know the first thing that pops into all of o...,"[https://www.suncor.com/en, https://www.suncor...",5
26804,"Well guys, I lost 90k today on SPY calls which...",[https://imgur.com/8zoihg7\n\n],1
7883,Stocks been up 46% in the past 5 days. They ma...,[https://nypost.com/2020/12/27/danimer],1
...,...,...,...
30079,I don't use the platform to trade but a fellow...,[https://preview.redd.it/5k3fq0h6gyd61.png],1
35388,[https://www.marketwatch.com/story/gamestop-an...,"[https://www.marketwatch.com/story/gamestop, h...",4
2985,"Hello, you rich beautiful degenerates. \n\nLet...","[https://heroicheartsproject.org, https://hunt...",9
17756,https://discord.gg/yc5772JExT,[https://discord.gg/yc5772JExT],1


# Tesseract OCR


In [75]:
class MemeOCR:
    def __init__(self):
        self._white_thresh = 240
        self._tmp_image_fname = './memeocr.jpg'
        self._tmp_txt_base = './memeocr'
        self._tmp_txt_fname = self._tmp_txt_base + '.txt'
        self._template_image = None
        self._keep_tmp_files = False

    def set_template(self, fname):
        self._template_image = self._read_image(fname)

    def recognize(self, fname):
        txt = None
        try:
            img = self._read_image(fname)
            self._thresh_words(img, self._template_image)
            self._exec_tesseract()
            txt = self._read_txt()
            self._delete_tmp_files()
        except:
            txt = ['']
        return txt

    def _read_image(self, fname):
        try:
            img = cv2.imread(fname)
        except IOError:
            img = None
        return img

    def _thresh_words(self, img, template):
#         if img is None:
#             return

#         for i in range(img.shape[0]):
#             for j in range(img.shape[1]):
#                 if all([elem >= self._white_thresh for elem in img[i][j]]):
#                     img[i][j] = (0, 0, 0)
#                 else:
#                     img[i][j] = (255, 255, 255)

        cv2.imwrite(self._tmp_image_fname, img)

    def _exec_tesseract(self):
        subprocess.run([
            'tesseract',
            '-l',
            'eng',
            self._tmp_image_fname,
            self._tmp_txt_base,
        ])

    def _read_txt(self):
#         try:
#             fr = open(self._tmp_txt_fname, encoding='utf-8')
#         except IOError:
#             return None
#         content = fr.read()
#         fr.close()

        with open(self._tmp_txt_fname, encoding='utf-8') as fr:
            content = fr.read()
            blocks = re.split(r'\n\n', content)
            lines = [re.sub(r'\s+', ' ', block) for block in blocks if block.strip()]
        return lines

    def _delete_tmp_files(self):
        if self._keep_tmp_files:
            return
        if os.path.exists(self._tmp_image_fname):
            os.remove(self._tmp_image_fname)
        if os.path.exists(self._tmp_txt_fname):
            os.remove(self._tmp_txt_fname)

In [6]:
def meme_ocr(file_path):
    ocr = MemeOCR()
    return ocr.recognize(file_path)

## Process images

In [52]:
def process_memes(url_lists):
    ocr = MemeOCR()
    retrived_strings = []
    
    for urls in url_lists:
        if len(urls) > 0:
            partial_results = []
            
            for url in urls:
                if is_url_image(url):
                    name = get_name(url)
                    url = correct_reddit_preview_url(url)
                    download_successful = download_image(url, name)
                    
                    if download_successful:
                        temp_path = path.join(temp_dir, name)
                        partial_results.append(ocr.recognize(temp_path))
                        delete_image(temp_path)
                    else:
                        partial_results.append([''])
                        
            retrived_strings.append(partial_results)
        else:
            retrived_strings.append([''])
                        
    return concat_results(retrived_strings)

In [8]:
def is_url_image(url):    
    mimetype,encoding = mimetypes.guess_type(url)
    return (mimetype and mimetype.startswith('image'))

In [9]:
def get_name(url):
    return url.split('/')[-1]

In [73]:
def download_image(url, name):
    try:
        request = requests.get(url, stream = True)
        status = request.status_code
    except:
        print('Connection error: ', url)
        status = -1
    
    if status == 200:
        with open(path.join(temp_dir, name), 'wb') as file:
            file.write(request.content)
            print('Image sucessfully Downloaded: ', name, ' From: ', url)
            return True
    else:
        print("Image Couldn't be retreived")
        return False

In [32]:
def delete_image(path_):
    path_type = path.normpath(path_)
    if path.exists(path_type):
        os.remove(path_type)

In [39]:
def correct_reddit_preview_url(url):
    return url.replace('preview.redd.it', 'i.redd.it')

In [47]:
def concat_results(string_lists):
    return [[' '.join(sentence) for sentence in list_] for list_ in string_lists]  

### Run

In [53]:
# dummy_data = data[['body', 'body_url', 'body_urls_count']].loc[data['body_urls_count'] > 0][:15]
# dummy_data

,body,body_url,body_urls_count
32591,$CBAT is going to be a great play further yet:...,[https://i.imgur.com/QF4i3h8.png],1
236,"I might not be able to read letters, but numbe...","[https://www.marketwatch.com/story/gamestop, h...",4
228,https://youtu.be/cRNypdYQoWk ELON MUSK SPEAKS ...,[https://youtu.be/cRNypdYQoWk],1
227,Hedge Fund Manipulation (BlackBerry) \n\nJim C...,"[https://youtu.be/vbEEdMFrW9w, https://youtu.b...",2
215,"So, media have been blaming us retards for wha...",[https://www.investopedia.com/terms/n/nakedsho...,1
211,I foresee this as being their last hope if the...,[https://www.msn.com/en],1
205,We have grown to the kind of size we only drea...,[http://twitter.com/wsbmod],1
192,There is a LOT of bad data flying around and i...,[https://www.reddit.com/r/wallstreetbets/comme...,1
188,"Like u/zjz said, this sub needs a fucking mega...",[https://twitter.com/stoolpresidente/status/13...,3
186,"Listen up retards.\n\nTime immemorial, the ric...",[https://youtu.be/_ZDEWVJan0s],1


In [80]:
# results = process_memes(dummy_data.body_url.values)
results = process_memes(data.body_url.values)

Image sucessfully Downloaded:  fphqf0xcorc61.png  From:  https://i.redd.it/fphqf0xcorc61.png
Image sucessfully Downloaded:  2lqwp6j4urc61.png  From:  https://i.redd.it/2lqwp6j4urc61.png
Image sucessfully Downloaded:  5zbvmohttrc61.png  From:  https://i.redd.it/5zbvmohttrc61.png
Image sucessfully Downloaded:  43ibx6arusc61.png  From:  https://i.redd.it/43ibx6arusc61.png
Image sucessfully Downloaded:  03dy5b4etsc61.png  From:  https://i.redd.it/03dy5b4etsc61.png
Image sucessfully Downloaded:  08w2w06husc61.png  From:  https://i.redd.it/08w2w06husc61.png
Image sucessfully Downloaded:  us5mx482atc61.png  From:  https://i.redd.it/us5mx482atc61.png
Image sucessfully Downloaded:  gg7lm3zj1uc61.png  From:  https://i.redd.it/gg7lm3zj1uc61.png
Image sucessfully Downloaded:  4p2nto0l1uc61.png  From:  https://i.redd.it/4p2nto0l1uc61.png
Image sucessfully Downloaded:  c7smltol1uc61.png  From:  https://i.redd.it/c7smltol1uc61.png
Image sucessfully Downloaded:  xv97r6hxztc61.png  From:  https://i.red

Image sucessfully Downloaded:  knmzkicqe9e61.png  From:  https://i.redd.it/knmzkicqe9e61.png
Image sucessfully Downloaded:  e05yn5pvt9e61.png  From:  https://i.redd.it/e05yn5pvt9e61.png
Image sucessfully Downloaded:  aaqwzskut9e61.png  From:  https://i.redd.it/aaqwzskut9e61.png
Image sucessfully Downloaded:  FAARPIg.png  From:  https://i.imgur.com/FAARPIg.png
Image sucessfully Downloaded:  3sV2GGG.png  From:  https://i.imgur.com/3sV2GGG.png
Image sucessfully Downloaded:  tHi3iJP.png  From:  https://i.imgur.com/tHi3iJP.png
Image sucessfully Downloaded:  b6nExyq.png  From:  https://i.imgur.com/b6nExyq.png
Image sucessfully Downloaded:  Hfq6e7l.png  From:  https://i.imgur.com/Hfq6e7l.png
Image sucessfully Downloaded:  rSAUfwb.jpg  From:  https://i.imgur.com/rSAUfwb.jpg
Image sucessfully Downloaded:  UO3iJXe.png  From:  https://i.imgur.com/UO3iJXe.png
Image sucessfully Downloaded:  tyEgAPB.png  From:  https://i.imgur.com/tyEgAPB.png
Image sucessfully Downloaded:  B8MdQcL.png  From:  https:

Image sucessfully Downloaded:  n6hyj8sbwwe61.png  From:  https://i.redd.it/n6hyj8sbwwe61.png
Image sucessfully Downloaded:  si1y9p3eywe61.jpg  From:  https://i.redd.it/si1y9p3eywe61.jpg
Image sucessfully Downloaded:  y6o34ax26xe61.jpg  From:  https://i.redd.it/y6o34ax26xe61.jpg
Image sucessfully Downloaded:  pv442cyi5xe61.jpg  From:  https://i.redd.it/pv442cyi5xe61.jpg
Image sucessfully Downloaded:  nugpxd3j6xe61.png  From:  https://i.redd.it/nugpxd3j6xe61.png
Image sucessfully Downloaded:  umca6tvhexe61.png  From:  https://i.redd.it/umca6tvhexe61.png
Image sucessfully Downloaded:  55c949ziexe61.png  From:  https://i.redd.it/55c949ziexe61.png
Image sucessfully Downloaded:  05n3xp1texe61.jpg  From:  https://i.redd.it/05n3xp1texe61.jpg
Image sucessfully Downloaded:  omgoo0mdixe61.png  From:  https://i.redd.it/omgoo0mdixe61.png
Image sucessfully Downloaded:  hqyf4qyxhwe61.jpg  From:  https://i.redd.it/hqyf4qyxhwe61.jpg
Image sucessfully Downloaded:  fj495vh52we61.jpg  From:  https://i.red

Image sucessfully Downloaded:  l6onj5wt7jd61.png  From:  https://i.redd.it/l6onj5wt7jd61.png
Image sucessfully Downloaded:  bnnoyx1o7jd61.jpg  From:  https://i.redd.it/bnnoyx1o7jd61.jpg
Image sucessfully Downloaded:  pcfxnxm6fjd61.png  From:  https://i.redd.it/pcfxnxm6fjd61.png
Image sucessfully Downloaded:  0nckd8u2gjd61.png  From:  https://i.redd.it/0nckd8u2gjd61.png
Image sucessfully Downloaded:  8osyp4jqdjd61.png  From:  https://i.redd.it/8osyp4jqdjd61.png
Image sucessfully Downloaded:  5w3qkp1jkjd61.png  From:  https://i.redd.it/5w3qkp1jkjd61.png
Image sucessfully Downloaded:  k10gelc9ojd61.png  From:  https://i.redd.it/k10gelc9ojd61.png
Image sucessfully Downloaded:  y2y2goggojd61.png  From:  https://i.redd.it/y2y2goggojd61.png
Image sucessfully Downloaded:  2s201j3ppjd61.jpg  From:  https://i.redd.it/2s201j3ppjd61.jpg
Image sucessfully Downloaded:  mz1ji1mm2kd61.png  From:  https://i.redd.it/mz1ji1mm2kd61.png
Image sucessfully Downloaded:  w63t8kwd1zc61.jpg  From:  https://i.red

Image sucessfully Downloaded:  tnlzwdozuhe61.jpg  From:  https://i.redd.it/tnlzwdozuhe61.jpg
Image sucessfully Downloaded:  jks5pd5e7ie61.png  From:  https://i.redd.it/jks5pd5e7ie61.png
Image sucessfully Downloaded:  lxj9n4k2cie61.png  From:  https://i.redd.it/lxj9n4k2cie61.png
Image sucessfully Downloaded:  r2ie5upueie61.png  From:  https://i.redd.it/r2ie5upueie61.png
Image sucessfully Downloaded:  A0R2dz2.jpg  From:  https://i.imgur.com/A0R2dz2.jpg
Image sucessfully Downloaded:  nQ6WcFN.jpg  From:  https://i.imgur.com/nQ6WcFN.jpg
Image sucessfully Downloaded:  40VDFnM.jpg  From:  https://i.imgur.com/40VDFnM.jpg
Image sucessfully Downloaded:  H7onSWz.jpg  From:  https://i.imgur.com/H7onSWz.jpg
Image sucessfully Downloaded:  wapFAGh.jpg  From:  https://i.imgur.com/wapFAGh.jpg
Image sucessfully Downloaded:  imHvK1O.jpg  From:  https://i.imgur.com/imHvK1O.jpg
Image sucessfully Downloaded:  bfCXbJj.jpg  From:  https://i.imgur.com/bfCXbJj.jpg
Image sucessfully Downloaded:  3R6pPLI.jpg  Fro

...

Image sucessfully Downloaded:  sj8ad5ozgkd61.png  From:  https://i.redd.it/sj8ad5ozgkd61.png
Image sucessfully Downloaded:  982au9eqjkd61.png  From:  https://i.redd.it/982au9eqjkd61.png
Image sucessfully Downloaded:  b5fksvdlnkd61.png  From:  https://i.redd.it/b5fksvdlnkd61.png
Image sucessfully Downloaded:  s6zas8zhgkd61.png  From:  https://i.redd.it/s6zas8zhgkd61.png
Image sucessfully Downloaded:  cn1mjfy1ikd61.png  From:  https://i.redd.it/cn1mjfy1ikd61.png
Image sucessfully Downloaded:  g2s1zpj5ikd61.png  From:  https://i.redd.it/g2s1zpj5ikd61.png
Image sucessfully Downloaded:  TZ6FzXu.png  From:  https://i.imgur.com/TZ6FzXu.png
Image sucessfully Downloaded:  7fpci3o1nkd61.png  From:  https://i.redd.it/7fpci3o1nkd61.png
Image sucessfully Downloaded:  tt5zodp6pkd61.png  From:  https://i.redd.it/tt5zodp6pkd61.png
Image sucessfully Downloaded:  qm81rd0dpkd61.png  From:  https://i.redd.it/qm81rd0dpkd61.png
Image sucessfully Downloaded:  8htb0scgpkd61.png  From:  https://i.redd.it/8htb0

Image sucessfully Downloaded:  o1kutmra5af61.png  From:  https://i.redd.it/o1kutmra5af61.png
Image sucessfully Downloaded:  a2vuu9dl7af61.jpg  From:  https://i.redd.it/a2vuu9dl7af61.jpg
Image sucessfully Downloaded:  qxty87mh9af61.png  From:  https://i.redd.it/qxty87mh9af61.png
Image sucessfully Downloaded:  06bvx0w4caf61.png  From:  https://i.redd.it/06bvx0w4caf61.png
Image sucessfully Downloaded:  hiaj77ngcaf61.png  From:  https://i.redd.it/hiaj77ngcaf61.png
Image sucessfully Downloaded:  pu70il3jcaf61.png  From:  https://i.redd.it/pu70il3jcaf61.png
Image sucessfully Downloaded:  wfcenh56daf61.png  From:  https://i.redd.it/wfcenh56daf61.png
Image sucessfully Downloaded:  hqi27yfndaf61.png  From:  https://i.redd.it/hqi27yfndaf61.png
Image sucessfully Downloaded:  83kqcn4ndaf61.png  From:  https://i.redd.it/83kqcn4ndaf61.png
Image sucessfully Downloaded:  0pqrfq0ciaf61.png  From:  https://i.redd.it/0pqrfq0ciaf61.png
Image sucessfully Downloaded:  o9xupt17laf61.png  From:  https://i.red

Image sucessfully Downloaded:  4479ak3okdc61.png  From:  https://i.redd.it/4479ak3okdc61.png
Image sucessfully Downloaded:  0u61nsroqdc61.jpg  From:  https://i.redd.it/0u61nsroqdc61.jpg
Image sucessfully Downloaded:  so6a8gy51ec61.png  From:  https://i.redd.it/so6a8gy51ec61.png
Image sucessfully Downloaded:  poptdji51ec61.png  From:  https://i.redd.it/poptdji51ec61.png
Image sucessfully Downloaded:  t5f3zdu6lec61.jpg  From:  https://i.redd.it/t5f3zdu6lec61.jpg
Image sucessfully Downloaded:  9fc3wh79iec61.jpg  From:  https://i.redd.it/9fc3wh79iec61.jpg
Image sucessfully Downloaded:  jejcfgdcqfc61.jpg  From:  https://i.redd.it/jejcfgdcqfc61.jpg
Image sucessfully Downloaded:  3zr9hentbgc61.png  From:  https://i.redd.it/3zr9hentbgc61.png
Image sucessfully Downloaded:  ri124qwrghc61.png  From:  https://i.redd.it/ri124qwrghc61.png
Image sucessfully Downloaded:  in4prw50ohc61.png  From:  https://i.redd.it/in4prw50ohc61.png
Image sucessfully Downloaded:  v5qz6bi0phc61.jpg  From:  https://i.red

Image sucessfully Downloaded:  m7mohkrwjwd61.png  From:  https://i.redd.it/m7mohkrwjwd61.png
Image sucessfully Downloaded:  6ss2szc4kwd61.png  From:  https://i.redd.it/6ss2szc4kwd61.png
Image sucessfully Downloaded:  rkft040rjwd61.png  From:  https://i.redd.it/rkft040rjwd61.png
Image sucessfully Downloaded:  841j294akwd61.png  From:  https://i.redd.it/841j294akwd61.png
Image sucessfully Downloaded:  i3d9eb27kwd61.png  From:  https://i.redd.it/i3d9eb27kwd61.png
Image sucessfully Downloaded:  yosmnjdlkwd61.png  From:  https://i.redd.it/yosmnjdlkwd61.png
Image sucessfully Downloaded:  ufmg7qbwfwd61.png  From:  https://i.redd.it/ufmg7qbwfwd61.png
Image sucessfully Downloaded:  fd07vulmlwd61.png  From:  https://i.redd.it/fd07vulmlwd61.png
Image sucessfully Downloaded:  kobhks31lwd61.png  From:  https://i.redd.it/kobhks31lwd61.png
Image sucessfully Downloaded:  fltxojoemwd61.png  From:  https://i.redd.it/fltxojoemwd61.png
Image sucessfully Downloaded:  8z3h340zlwd61.png  From:  https://i.red

Image sucessfully Downloaded:  7e9kalgouwd61.png  From:  https://i.redd.it/7e9kalgouwd61.png
Image sucessfully Downloaded:  pxh34wzmuwd61.png  From:  https://i.redd.it/pxh34wzmuwd61.png
Image sucessfully Downloaded:  ahxy9zg2vwd61.png  From:  https://i.redd.it/ahxy9zg2vwd61.png
Image sucessfully Downloaded:  9ebc2gyb1xd61.png  From:  https://i.redd.it/9ebc2gyb1xd61.png
Image sucessfully Downloaded:  0TwYJQE.jpg  From:  https://i.imgur.com/0TwYJQE.jpg
Image sucessfully Downloaded:  xbaizei2wwd61.png  From:  https://i.redd.it/xbaizei2wwd61.png
Image sucessfully Downloaded:  6ldc7cf6wwd61.png  From:  https://i.redd.it/6ldc7cf6wwd61.png
Image sucessfully Downloaded:  c84pujekwwd61.png  From:  https://i.redd.it/c84pujekwwd61.png
Image sucessfully Downloaded:  nd3999qtwwd61.png  From:  https://i.redd.it/nd3999qtwwd61.png
Image sucessfully Downloaded:  g8r454ppxwd61.png  From:  https://i.redd.it/g8r454ppxwd61.png
Image sucessfully Downloaded:  tnupjfxswwd61.png  From:  https://i.redd.it/tnupj

In [81]:
import pickle 

with open('meme_ocr.pkl', 'wb') as file:
    pickle.dump(results, file)

In [82]:
with open('meme_ocr.pkl', 'rb') as file:
    test = pickle.load(file)
    
test

[[''],
 [''],
 [''],
 [''],
 [''],
 ['ASSETS Current assets: Cash and cash equivalents Restricted cash Receivables, net Merchandise inventories, net Prepaid expenses and other current assets Assets held-for-sale Total current assets Property and equipment, net Operating lease right-of-use assets Deferred income taxes Long-term restricted cash (Other noncurrent assets Total assets, LIABILITIES AND STOCKHOLDERS’ EQUITY Current liabilities ‘Accounts payable Accrued liabilities and other current liabilities Current portion of operating lease liabilities ‘Short-term debt, including current portion of long-term debt, net Borrowings under revolving line of credit Total current abilities Long-term debt, net Operating lease liabilities Other long-term liabilities Total liabilities Commitments and contingencies (Note 7) ‘Stockholders’ equity: Class A common stock — § .001 par value; 300 shares authorized: 65.2 , 67.9 and 64.3 shares issued and outstanding Additional paid-in capital Accumulated o

In [84]:
print(data.shape[0])
print(len(test))

38908
38908
